# Analysis of correlations of gold, silver and other datasets

## Setup

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
# Get gold and silver dfs and several other dfs
df_au = pd.read_csv("../data/AU.csv")
df_ag = pd.read_csv("../data/AG.csv")

df_au_fut = pd.read_csv("../data/AU_FUTURES.csv")
df_ag_fut = pd.read_csv("../data/AG_FUTURES.csv")
df_au_min = pd.read_csv("../data/AU_MINERS_ETF.csv")
df_ag_min = pd.read_csv("../data/AG_MINERS_ETF.csv")
df_au_res = pd.read_csv("../data/AU_WORLD_RESERVES_OZ_M.csv")

df_usgdp = pd.read_csv("../data/US_REAL_GDP_Q.csv")
df_cpi = pd.read_csv("../data/US_CPI_M.csv")
df_fed = pd.read_csv("../data/FED_FUNDS_EFFECTIVE_RATE_M.csv")
df_us10y = pd.read_csv("../data/US_10Y_TREASURY_YIELD.csv")
df_dxy = pd.read_csv("../data/DXY.csv")

df_sp500 = pd.read_csv("../data/SP500.csv")
df_vix = pd.read_csv("../data/VIX.csv")
df_btc = pd.read_csv("../data/BTC.csv")

In [ ]:
# Append the CPI estimate for April of 2024 to the CPI df
new_row = pd.DataFrame({"year_month": ["2024-04"], "rate": [3.4]})
df_cpi = pd.concat([df_cpi, new_row], ignore_index=True)

In [ ]:
# Set date as index for all dfs
for df in df_au, df_ag, df_au_fut, df_ag_fut, df_au_min, df_ag_min, df_au_res, df_usgdp, df_cpi, df_fed, df_us10y, df_dxy, df_sp500, df_vix, df_btc:
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df.set_index(date_col, inplace=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

In [ ]:
# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

## Datasets basic info

*Only the ones that weren't presented yet.*

### Gold futures

In [ ]:
# First and last entries
pd.concat([df_au_fut.head(1), df_au_fut.tail(1)]).T

In [ ]:
df_au_fut.describe().T

### Silver futures

In [ ]:
# First and last entries
pd.concat([df_ag_fut.head(1), df_ag_fut.tail(1)]).T

In [ ]:
df_ag_fut.describe().T

### Gold miners ETF

In [ ]:
# First and last entries
pd.concat([df_au_min.head(1), df_au_min.tail(1)]).T

In [ ]:
df_au_min.describe().T

### Silver miners ETF

In [ ]:
# First and last entries
pd.concat([df_ag_min.head(1), df_ag_min.tail(1)]).T

In [ ]:
df_ag_min.describe().T

### Gold world reserves in central banks in troy ounces

In [ ]:
# First and last entries
pd.concat([df_au_res.head(1), df_au_res.tail(1)]).T

In [ ]:
df_au_res.describe().T

### US real GDP 2017 prices

In [ ]:
# First and last entries
pd.concat([df_usgdp.head(1), df_usgdp.tail(1)]).T

In [ ]:
df_usgdp.describe().T

### US 10 year treasury bonds yield

In [ ]:
# First and last entries
pd.concat([df_us10y.head(1), df_us10y.tail(1)]).T

In [ ]:
df_us10y.describe().T

### Dollar index (DXY)

In [ ]:
# First and last entries
pd.concat([df_dxy.head(1), df_dxy.tail(1)]).T

In [ ]:
df_dxy.describe().T

### S&P 500 index

In [ ]:
# First and last entries
pd.concat([df_sp500.head(1), df_sp500.tail(1)]).T

In [ ]:
df_sp500.describe().T

*The "open" metric is unreliable, often showing as 0, which is incorrect. Therefore, we'll use the HLC mean instead of the OHLC mean.*

### Volatility index (VIX)

In [ ]:
# First and last entries
pd.concat([df_vix.head(1), df_vix.tail(1)]).T

In [ ]:
df_vix.describe().T

### Bitcoin price (BTC)

In [ ]:
# First and last entries
pd.concat([df_btc.head(1), df_btc.tail(1)]).T

In [ ]:
df_btc.describe().T

## Preparing all data into a combined df

In [ ]:
# Get OHLC average as price from dfs
for df in df_au_fut, df_ag_fut, df_au_min, df_ag_min, df_dxy, df_vix, df_btc:
    df["price"] = df[["open", "high", "low", "close"]].mean(axis=1)
    df.drop(columns=["open", "high", "low", "close"], inplace=True)

# Get HLC average as price from S&P 500 df (due to zero values in the open col)
df_sp500["price"] = df_sp500[["high", "low", "close"]].mean(axis=1)
df_sp500.drop(columns=["open", "high", "low", "close"], inplace=True)

In [ ]:
# Resample date of non-daily dfs 
df_au_res, df_usgdp, df_cpi, df_fed = [df.resample("D").ffill() for df in [df_au_res, df_usgdp, df_cpi, df_fed]]

In [ ]:
# Get 1-year volatility of gold and silver
for df, name in [(df_au, "au"), (df_ag, "ag")]:
    df["price_change"] = df["price"].pct_change()
    df[f"{name}_volatility"] = df["price_change"].rolling(window=252).std()
    df.drop(columns=["price_change"], inplace=True)

In [ ]:
# Rename dfs cols
df_au.rename(columns={"price": "au"}, inplace=True)
df_ag.rename(columns={"price": "ag"}, inplace=True)
df_au_fut.rename(columns={"price": "au_futures"}, inplace=True)
df_ag_fut.rename(columns={"price": "ag_futures"}, inplace=True)
df_au_min.rename(columns={"price": "au_minners"}, inplace=True)
df_ag_min.rename(columns={"price": "ag_minners"}, inplace=True)
df_au_res.rename(columns={"amount": "au_reserves"}, inplace=True)
df_usgdp.rename(columns={"value": "us_real_gdp"}, inplace=True)
df_cpi.rename(columns={"rate": "us_cpi"}, inplace=True)
df_fed.rename(columns={"rate": "fed_rate"}, inplace=True)
df_us10y.rename(columns={"yield": "us_10y_yield"}, inplace=True)
df_dxy.rename(columns={"price": "dxy"}, inplace=True)
df_sp500.rename(columns={"price": "sp500"}, inplace=True)
df_vix.rename(columns={"price": "vix"}, inplace=True)
df_btc.rename(columns={"price": "btc"}, inplace=True)

In [ ]:
# Combine all in one df (null values do not need to be dropped because they are ignored in the calculations, this way no dataset will be shortened)
df_all = pd.concat([df_au, df_ag, df_au_fut, df_ag_fut, df_au_min, df_ag_min, df_au_res, df_usgdp, df_cpi, df_fed, df_us10y, df_dxy, df_sp500, df_vix, df_btc], axis=1)

## Covariance matrix

In [ ]:
# Get covariance matrix
df_cov_matrix = df_all.cov()
df_cov_matrix

**Not very insightful. We must check correlations.**

## Correlation matrix (Pearson method)

In [ ]:
# Get correlation matrix
df_corr_matrix = df_all.corr().round(3)
df_corr_matrix

### Heatmap

In [ ]:
plt.figure(figsize=(13, 8))

sns.heatmap(data=df_corr_matrix, annot=True, cmap="coolwarm", center=0)

plt.title("Correlation matrix heat map")

save_chart_as_png("4_AU_AG_correlations_matrix")

**Key takeaways:**
- The spot and futures markets have a perfect correlation, as anticipated.
- Gold and silver show a strong correlation, including their respective yearly volatilities.
- Gold is strongly correlated with the S&P 500 and US real GDP, while silver has a moderate correlation with these indicators.
- Gold has a strong correlation with Bitcoin, whereas silver shows no correlation.
- Gold exhibits a moderate negative correlation with the Fed funds rate and the US 10-year Treasury yield, with silver showing a slightly weaker negative correlation.
- Both metals have a weak to moderate negative correlation with the dollar index.
- Gold shows no correlation with mining stocks, in contrast to silver, which has a strong correlation, particularly with its miners.
- There is no correlation between central banks' gold reserves and the precious metals.
- Both metals have little to no correlation with US inflation and the volatility index; however, the metals' volatilities, especially gold's, are moderately correlated with these indicators.

### Top ten correlations

**Gold price**

In [ ]:
# Top 10 (abs) corr with gold price
df_sorted_corr = df_corr_matrix.sort_values(by="au", key=lambda x: x.abs(), ascending=False)
df_sorted_corr.drop("au", inplace=True)
df_sorted_corr = df_sorted_corr.iloc[:10][["au"]]
df_sorted_corr.T

In [ ]:
# Get type of correlation and absolute values for chart
df_sorted_corr["type"] = df_sorted_corr["au"].apply(lambda x: "Negative" if x < 0 else "Positive")
df_sorted_corr["au"] = df_sorted_corr["au"].abs()

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_sorted_corr, x="au", y=df_sorted_corr.index, hue="type", palette={"Positive": "goldenrod", "Negative": "orangered"})

plt.title("Top 10 gold price correlations")
plt.xlabel("")
plt.ylabel("")
plt.legend(title="Correlation type")

**Gold 1-year volatility**

In [ ]:
# Top 10 (abs) corr with gold 1-year volatility
df_sorted_corr = df_corr_matrix.sort_values(by="au_volatility", key=lambda x: x.abs(), ascending=False)
df_sorted_corr.drop("au_volatility", inplace=True)
df_sorted_corr = df_sorted_corr.iloc[:10][["au_volatility"]]
df_sorted_corr.T

In [ ]:
# Get type of correlation and absolute values for chart
df_sorted_corr["type"] = df_sorted_corr["au_volatility"].apply(lambda x: "Negative" if x < 0 else "Positive")
df_sorted_corr["au_volatility"] = df_sorted_corr["au_volatility"].abs()

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_sorted_corr, x="au_volatility", y=df_sorted_corr.index, hue="type", palette={"Positive": "chocolate", "Negative": "red"})

plt.title("Top 10 gold price yearly volatility correlations")
plt.xlabel("")
plt.ylabel("")
plt.legend(title="Correlation type")

**Silver price**

In [ ]:
# Top 10 (abs) corr with silver price
df_sorted_corr = df_corr_matrix.sort_values(by="ag", key=lambda x: x.abs(), ascending=False)
df_sorted_corr.drop("ag", inplace=True)
df_sorted_corr = df_sorted_corr.iloc[:10][["ag"]]
df_sorted_corr.T

In [ ]:
# Get type of correlation and absolute values for chart
df_sorted_corr["type"] = df_sorted_corr["ag"].apply(lambda x: "Negative" if x < 0 else "Positive")
df_sorted_corr["ag"] = df_sorted_corr["ag"].abs()

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_sorted_corr, x="ag", y=df_sorted_corr.index, hue="type", palette={"Positive": "silver", "Negative": "lightcoral"})

plt.title("Top 10 silver price correlations")
plt.xlabel("")
plt.ylabel("")
plt.legend(title="Correlation type")

**Silver 1-year volatility**

In [ ]:
# Top 10 (abs) corr with silver 1-year volatility
df_sorted_corr = df_corr_matrix.sort_values(by="ag_volatility", key=lambda x: x.abs(), ascending=False)
df_sorted_corr.drop("ag_volatility", inplace=True)
df_sorted_corr = df_sorted_corr.iloc[:10][["ag_volatility"]]
df_sorted_corr.T

In [ ]:
# Get type of correlation and absolute values for chart
df_sorted_corr["type"] = df_sorted_corr["ag_volatility"].apply(lambda x: "Negative" if x < 0 else "Positive")
df_sorted_corr["ag_volatility"] = df_sorted_corr["ag_volatility"].abs()

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_sorted_corr, x="ag_volatility", y=df_sorted_corr.index, hue="type", palette={"Positive": "rosybrown", "Negative": "red"})

plt.title("Top 10 silver price yearly volatility correlations")
plt.xlabel("")
plt.ylabel("")
plt.legend(title="Correlation type")